In [201]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import requests
import os

In [202]:
def ask_user():
    place = input("Please enter the name of the place (city, State) you want to search restaurants in (e.g. \"Fremont, CA\"): ")
    lst = [x.strip() for x in place.split(',')]
    if len(lst[0].split())>1:
        lst[0] ='+'.join(lst[0].split())
        return lst
    else:
        return lst

In [204]:
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

# Read the HTML from the URL and pass on to BeautifulSoup
baseurl = 'https://www.yelp.com/search?find_desc=Restaurants&find_loc='
lst = ask_user()
url = baseurl +lst[0]+',+'+lst[1]
print("Opening the page", url)
uh= urllib.request.urlopen(url, context=ctx)
print("HTTP status",uh.getcode())
html =uh.read()
print(f"Reading done. Total {len(html)} characters read.")

Please enter the name of the place (city, State) you want to search restaurants in (e.g. "Fremont, CA"): San Jose, CA
Opening the page https://www.yelp.com/search?find_desc=Restaurants&find_loc=San+Jose,+CA
HTTP status 200
Reading done. Total 293664 characters read.


In [205]:
soup = BeautifulSoup(html, 'html.parser')

In [208]:
restaurant_names = []
for span in soup.find_all('span'):
    if 'class' in span.attrs:
        if span.attrs['class']==['indexed-biz-name']:
            restaurant_names.append(span.contents[1].get_text())

In [209]:
restaurant_names

['The Funny Farm',
 'Cajun Bistro 7',
 'Vietnoms',
 'Pot Belly Ramen',
 'Holy Cannoli Cafe',
 'The Table',
 'TAOB Pit Stop',
 'Gen Korean BBQ House',
 'Back A Yard',
 'Mesquite and Oak']

In [210]:
restaurants_links=[]
for a in soup.find_all('a'):
    if 'class' in a.attrs:
        #print(a.attrs)
        if a.attrs['class']==['js-analytics-click']:
            restaurants_links.append(a.attrs['href'])
_=restaurants_links.pop(0)

In [211]:
for i in range(len(restaurants_links)):
    link='https://yelp.com'+restaurants_links[i]
    restaurants_links[i]=link

In [225]:
restaurants_links

['https://yelp.com/biz/the-funny-farm-san-jose?frvs=True&osq=Restaurants',
 'https://yelp.com/biz/cajun-bistro-7-san-jose-2?frvs=True&osq=Restaurants',
 'https://yelp.com/biz/vietnoms-san-jose?frvs=True&osq=Restaurants',
 'https://yelp.com/biz/pot-belly-ramen-san-jose?frvs=True&osq=Restaurants',
 'https://yelp.com/biz/holy-cannoli-cafe-san-jose-2?frvs=True&osq=Restaurants',
 'https://yelp.com/biz/the-table-san-jose?frvs=True&osq=Restaurants',
 'https://yelp.com/biz/taob-pit-stop-san-jose?frvs=True&osq=Restaurants',
 'https://yelp.com/biz/gen-korean-bbq-house-san-jose?frvs=True&osq=Restaurants',
 'https://yelp.com/biz/back-a-yard-san-jose?frvs=True&osq=Restaurants',
 'https://yelp.com/biz/mesquite-and-oak-san-jose?frvs=True&osq=Restaurants']

In [226]:
df=pd.DataFrame(data={'Link':restaurants_links,'Name':restaurant_names})

In [227]:
df

,Link,Name
0,https://yelp.com/biz/the-funny-farm-san-jose?f...,The Funny Farm
1,https://yelp.com/biz/cajun-bistro-7-san-jose-2...,Cajun Bistro 7
2,https://yelp.com/biz/vietnoms-san-jose?frvs=Tr...,Vietnoms
3,https://yelp.com/biz/pot-belly-ramen-san-jose?...,Pot Belly Ramen
4,https://yelp.com/biz/holy-cannoli-cafe-san-jos...,Holy Cannoli Cafe
5,https://yelp.com/biz/the-table-san-jose?frvs=T...,The Table
6,https://yelp.com/biz/taob-pit-stop-san-jose?fr...,TAOB Pit Stop
7,https://yelp.com/biz/gen-korean-bbq-house-san-...,Gen Korean BBQ House
8,https://yelp.com/biz/back-a-yard-san-jose?frvs...,Back A Yard
9,https://yelp.com/biz/mesquite-and-oak-san-jose...,Mesquite and Oak


In [215]:
reviews={}
for i in range(10):
    print(f"Gathering top reviews on {restaurant_names[i]} now...")
    review_text=[]
    url=restaurants_links[i]
    uh= urllib.request.urlopen(url, context=ctx)
    html =uh.read()
    soup = BeautifulSoup(html, 'html.parser')
    for p in soup.find_all('p'):
        if 'itemprop' in p.attrs:
            if p.attrs['itemprop']=='description':
                text=p.get_text().strip()
                review_text.append(text)
    reviews[str(restaurant_names[i])]=review_text

Gathering top reviews on The Funny Farm now...
Gathering top reviews on Cajun Bistro 7 now...
Gathering top reviews on Vietnoms now...
Gathering top reviews on Pot Belly Ramen now...
Gathering top reviews on Holy Cannoli Cafe now...
Gathering top reviews on The Table now...
Gathering top reviews on TAOB Pit Stop now...
Gathering top reviews on Gen Korean BBQ House now...
Gathering top reviews on Back A Yard now...
Gathering top reviews on Mesquite and Oak now...


In [221]:
for r in reviews['The Table']:
    print(r)
    print("="*100)

Service (5/5): Our waitress made sure our bottomless mimosas were constantly full lol. It was pretty busy but the servers were on it! Every server was willing to cater to any of our needs. 

Ambiance (5/5): Dog friendly, open windows, cute decor, and an outdoor area. 

Food (5/5): B--M-B. We ordered the chilaquiles, pork fried steak, salmon Benedict, and the loco moco. I didn't try the first two dishes I mentioned but out of the Benedict and loco moco my favorite was definitely the latter. If you like island food I highly recommend. 

Price (4/5): My advice is to eat slow. The price was definitely worth its quality but the quantity isn't very big. My group was pretty full but it's because of the mimosas and we ordered monkey bread and beignets.
Our experience at The Table was OK, but I had higher expectations for a popular brunch place with 2,000+ reviews.  

I knew the wait was going to be a nightmare for a weekend brunch, and they don't take reservations.  However, luckily I picked u